In [31]:
import tensorflow as tf
import numpy as np
import sklearn

In [32]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount=True)
import os
os.chdir('/content/Drive/My Drive/DataMining_Project')

Mounted at /content/Drive


# Model Definition

## Downblock

In [33]:
"""
  A downsample block consists of a 
    - Convolutional Layer
    - Batch Norm Layer
    - Activation Function (This can be experimented with as necessary)

"""
def downsample(stride = 2, filters = 32, dropout = 0):
  result = tf.keras.Sequential()
  result.add(tf.keras.layers.Conv2D(kernel_size = 2, strides = stride,  filters = filters,  kernel_regularizer='l2'))
  result.add(tf.keras.layers.BatchNormalization())
  result.add(tf.keras.layers.Dropout(dropout))
  result.add(tf.keras.layers.ReLU())
  return result

## Model Architectures

### Shallow

In [34]:
def shallow_CNN(model_type = None, dropout = 0,  in_shape = (256,256,3)):
  assert model_type in ["disease", "plant"]
  current_image = 256
  downsample_blocks = []
  #Determine how many cnn blocks are necessary
  

  #Create the input
  input = tf.keras.Input(shape =  in_shape )
  next = input
  next = tf.keras.layers.Conv2D(kernel_size = 256, strides = 1,  filters = 256,  kernel_regularizer='l2')(next)
  next = tf.keras.layers.BatchNormalization()(next)
  next = tf.keras.layers.ReLU()(next)
  #Get a fully connected layer : This will be used to extract the input for the random forest and the SVM models
  next = tf.keras.layers.Flatten()(next)
  next = tf.keras.layers.Dense(100,  kernel_regularizer='l2')(next)
  
  #Add another dense layer to resize to number of classes
  if model_type == "plant":
    next = tf.keras.layers.Dense(11)(next)
  else:
    next = tf.keras.layers.Dense(38)(next)
  
  #Turn the final layer into a probability layer to get the predictions
  final = tf.keras.layers.Softmax()(next)  
  return tf.keras.Model(inputs = input, outputs = final)

### Basic

In [35]:
def basic_CNN(model_type = None, dropout = 0,  in_shape = (256,256,3)):
  assert model_type in ["disease", "plant"]
  print(in_shape)
  current_image = 256
  downsample_blocks = []
  #Determine how many cnn blocks are necessary
  while current_image > 1:
    
    downsample_blocks.append(downsample(stride = 2, filters = 128, dropout = dropout))
    current_image = current_image/2

  #Create the input
  input = tf.keras.Input(shape =  in_shape )
  next = input
  #Pass it through each CNN layer
  for downblock in downsample_blocks:
    next = downblock(next)
    
  #Get a fully connected layer : This will be used to extract the input for the random forest and the SVM models
  next = tf.keras.layers.Flatten()(next)
  next = tf.keras.layers.Dense(100,  kernel_regularizer='l2')(next)
  
  #Add another dense layer to resize to number of classes
  if model_type == "plant":
    next = tf.keras.layers.Dense(11)(next)
  else:
    next = tf.keras.layers.Dense(38)(next)
  
  #Turn the final layer into a probability layer to get the predictions
  final = tf.keras.layers.Softmax()(next)  
  return tf.keras.Model(inputs = input, outputs = final)


### Small

In [36]:
def small_CNN(model_type = None, dropout = 0,  in_shape = (256,256,3)):
  assert model_type in ["disease", "plant"]
  current_image = 256
  downsample_blocks = []
  #Determine how many cnn blocks are necessary
  downsample_blocks = [
                       downsample(stride = 2, filters = 8, dropout = dropout),   #output shape : [None, 128,128, 8]
                       downsample(stride = 2, filters = 16, dropout = dropout),   #output shape : [None, 64, 64, 16]
                       downsample(stride = 2, filters = 32, dropout = dropout),   #output shape : [None, 32,32, 32]
                       downsample(stride = 2, filters = 64, dropout = dropout),   #output shape : [None, 16,16, 64]
                       downsample(stride = 2, filters = 128, dropout = dropout),   #output shape : [None, 8,8, 128]
                       downsample(stride = 2, filters = 128, dropout = dropout),   #output shape : [None, 4,4, 128]
                       downsample(stride = 2, filters = 128, dropout = dropout),   #output shape : [None, 2,2, 128]
                       downsample(stride = 2, filters = 128, dropout = dropout),   #output shape : [None, 1,1, 128]
                      ]

  #Create the input
  input = tf.keras.Input(shape =  in_shape )
  next = input
  #Pass it through each CNN layer
  for downblock in downsample_blocks:
    next = downblock(next)

  #Get a fully connected layer : This will be used to extract the input for the random forest and the SVM models
  next = tf.keras.layers.Flatten()(next)
  next = tf.keras.layers.Dense(100,  kernel_regularizer='l2')(next)
  
  if model_type == "plant":
    next = tf.keras.layers.Dense(11)(next)
  else:
    next = tf.keras.layers.Dense(38)(next)
  
  final = tf.keras.layers.Softmax()(next)  
  
  return tf.keras.Model(inputs = input, outputs = final)

### Tiny

In [37]:
def tiny_CNN(model_type = None, dropout = 0,  in_shape = (256,256,3)):
  current_image = 256
  downsample_blocks = []
  #Determine how many cnn blocks are necessary
  downsample_blocks = [
                       downsample(stride = 4, filters = 4, dropout = dropout),    #output shape : [None, 64, 64, 4]
                       downsample(stride = 4, filters = 8, dropout = dropout),   #output shape : [None, 16, 16, 8]
                       downsample(stride = 4, filters = 16, dropout = dropout),   #output shape : [None, 4, 4, 16]
                       downsample(stride = 4, filters = 32, dropout = dropout),   #output shape : [None, 1, 1, 32]
                      ]

  #Create the input
  input = tf.keras.Input(shape = in_shape)
  next = input
  #Pass it through each CNN layer
  for downblock in downsample_blocks:
    next = downblock(next)
    #print(next.shape)

  #Get a fully connected layer : This will be used to extract the input for the random forest and the SVM models
  next = tf.keras.layers.Flatten()(next)
  next = tf.keras.layers.Dense(100,  kernel_regularizer='l2')(next)
  #print(next.shape)
  #Add another dense layer to resize to number of classes
  if model_type == "plant":
    next = tf.keras.layers.Dense(11)(next)
  else:
    next = tf.keras.layers.Dense(38)(next)
  #print(next.shape)
  #Turn the final layer into a probability layer to get the predictions
  final = tf.keras.layers.Softmax()(next)  
  #final = tf.math.argmax(final, axis = -1).int64()  
  #print(type(final))
  return tf.keras.Model(inputs = input, outputs = final)

# Create Dataset

In [38]:
def create_dataset(path, batch_size):
  dataset_location = path
  if "processed" in path:
    train_data = tf.keras.utils.image_dataset_from_directory(dataset_location, 
                                                            labels = "inferred", 
                                                            label_mode = "categorical", 
                                                            image_size = (256,256),
                                                            subset =  "training",
                                                            validation_split = .2,
                                                            color_mode = "rgba",
                                                            batch_size = batch_size,
                                                            shuffle = True,
                                                            seed = 1)
    validation_data = tf.keras.utils.image_dataset_from_directory(dataset_location, 
                                                              labels = "inferred", 
                                                              label_mode = "categorical", 
                                                              image_size = (256,256),
                                                              subset =  "validation",
                                                              validation_split = .2,
                                                              batch_size = batch_size,
                                                              color_mode = "rgba",
                                                              shuffle = True,
                                                              seed = 1)
    return train_data, validation_data
  train_data = tf.keras.utils.image_dataset_from_directory(dataset_location, 
                                                            labels = "inferred", 
                                                            label_mode = "categorical", 
                                                            image_size = (256,256),
                                                            subset =  "training",
                                                            validation_split = .2,
                                                            batch_size = batch_size,
                                                            shuffle = True,
                                                            seed = 1)
  validation_data = tf.keras.utils.image_dataset_from_directory(dataset_location, 
                                                            labels = "inferred", 
                                                            label_mode = "categorical", 
                                                            image_size = (256,256),
                                                            subset =  "validation",
                                                            validation_split = .2,
                                                            batch_size = batch_size,
                                                            shuffle = True,
                                                            seed = 1)
  return train_data, validation_data

In [39]:
def create_class_weights(path):
  counts = []    
  old_location = path
  for i,dir in enumerate(os.walk(old_location)):
      
      if dir[0] == path:
        continue
      counts.append(len(os.listdir(dir[0])))
  max_count = max(counts)
  class_weights = max_count / np.array(counts )
  weights = {}
  for i, weight in enumerate(class_weights):
    weights[i] = weight
  return weights 


# Train Model

In [40]:
def train_model(model, data_path, log_dir, ckpt_dir, batch_size):
  train_data, validation_data = create_dataset(data_path, batch_size = batch_size)
  class_weights = create_class_weights(data_path)
  tf_callbacks = tf.keras.callbacks.TensorBoard(log_dir = log_dir, update_freq = 1 )
  tf_modelCkpt = tf.keras.callbacks.ModelCheckpoint(ckpt_dir, monitor="val_loss", save_best_only=True,  save_weights_only=False, mode="auto", save_freq='epoch', initial_value_threshold=None)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss = tf.keras.losses.CategoricalCrossentropy(), metrics = ["accuracy", "mae"])
  model.fit(train_data, epochs = 150, validation_freq = 2, validation_data = validation_data, class_weight = class_weights, callbacks = [tf_callbacks, tf_modelCkpt])

## Train Shallow

In [41]:
train_model(shallow_CNN(model_type = "disease", dropout = 0, in_shape = (256,256,4)), "data/processed",'saved_models/shallow/logs', "saved_models/shallow/checkpoints/", 32 )


KeyboardInterrupt: ignored

# Coarse Search for Optimal

In [ ]:
models = [basic_CNN, small_CNN, tiny_CNN]
model_name = ["basic", "small", "tiny"]
dropouts = [0,.05,.1,.15]
for m, mn in zip(models, model_name):
  for d in dropouts:
    train_model(m(model_type = "disease", dropout = d, in_shape = (256,256,4)), "data/processed" ,f'saved_models/processed/{mn}/{d}/logs', f"saved_models/processed/{mn}/{d*100}/checkpoints/", 32 )
    train_model(m(model_type = "disease", dropout = d, in_shape = (256,256,3)), "data/raw" ,f'saved_models/raw/{mn}/{d}/logs', f"saved_models/raw/{mn}/{d*100}/checkpoints/", 32 )

# Transfer Learning

In [ ]:
def resnet101_model(in_shape = (256,256,3)):
  model = tf.keras.models.Sequential()
  input = tf.keras.Input(shape = in_shape)
  resnet = tf.keras.applications.ResNet101V2(input_tensor = input)
  model.add(resnet)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Dense(38, activation = "softmax"))
  return model

def resnet50_model(in_shape = (256,256,3)):
  model = tf.keras.models.Sequential()
  input = tf.keras.Input(shape = in_shape)
  resnet = tf.keras.applications.ResNet50(input_tensor = input)
  model.add(resnet)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Dense(38, activation = "softmax"))
  return model
  
def mobilenet_model(in_shape = (256,256,3)):
  model = tf.keras.models.Sequential()
  input = tf.keras.Input(shape = in_shape)
  resnet = tf.keras.applications.MobileNetV2(input_tensor = input)
  model.add(resnet)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Dense(38, activation = "softmax"))
  return model

def inception_model(in_shape = (256,256,3)):
  model = tf.keras.models.Sequential()
  input = tf.keras.Input(shape = in_shape)
  resnet = tf.keras.applications.InceptionV3(input_tensor = input)
  model.add(resnet)
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(100))
  model.add(tf.keras.layers.Dense(38, activation = "softmax"))
  return model


In [ ]:
train_model(resnet50_model(), "data/raw" ,f'saved_models/ResNet50/logs', f"saved_models/ResNet50/checkpoints/", 32 )

In [ ]:

train_model(resnet101_model, "data/raw" ,f'saved_models/ResNet101/logs', f"saved_models/ResNet101/checkpoints/", 32 )

In [ ]:

train_model(inception_model, "data/raw" ,f'saved_models/InceptionV3/logs', f"saved_models/InceptionV3/checkpoints/", 32 )

In [ ]:

train_model(mobilenet_model, "data/raw" ,f'saved_models/MobileNet/logs', f"saved_models/MobileNet/checkpoints/", 32 )

# SVM and RF with Best Model

In [ ]:
model = tf.keras.models.load_model('saved_models/ResNet101V2_continued/checkpoints')

In [ ]:
disease_list = []
for i,dir in enumerate(os.walk("data/raw")):
  if dir[0] == "data/raw":
    continue
  disease_list.append(dir[0][dir[0].rfind('/')+1:])
disease_list.sort()
labels_dict = {}
for x in range(len(disease_list)):
  labels_dict[x] = disease_list[x]


In [ ]:
path = 'data/raw'
train_data, validation_data = create_dataset(path, batch_size = 32)

In [ ]:
class_weights = create_class_weights("data/raw")
print(class_weights)
rf_weights = {}
for x, y in class_weights.items():
  rf_weights[labels_dict[x]] = y
rf_weights

In [ ]:
val_x = []
val_y = []

intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)


for x in validation_data:
  for row in x[1]:
    val_y.append(labels_dict[int(tf.math.argmax(row))])
  temp = intermediate_layer_model.predict(x[0])
  temp = np.array(temp)
  for row in temp:
    temp_row = row.reshape(100)
    val_x.append(row)

train_x = []
train_y = []

for x in train_data:
  for row in x[1]:
    train_y.append(labels_dict[int(tf.math.argmax(row))])
  temp = intermediate_layer_model.predict(x[0])
  temp = np.array(temp)
  for row in temp:
    temp_row = row.reshape(100)
    train_x.append(row)
  

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)
# Train the model on training data
#rf.fit(train_features, train_labels);
rf.fit(train_x, train_y)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = rf.predict(val_x)
print(confusion_matrix(val_y, predictions))
print(classification_report(val_y,predictions))

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(train_x, train_y)

In [ ]:
predictions = svclassifier.predict(val_x)
print(confusion_matrix(val_y, predictions))
print(classification_report(val_y,predictions))